In [1]:
pip install pyserini==0.11.0.0

     |████████████████████████████████| 67.1MB 55kB/s 
     |████████████████████████████████| 2.1MB 40.8MB/s 
     |████████████████████████████████| 1.1MB 39.5MB/s 
     |████████████████████████████████| 7.9MB 866kB/s 
     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 3.3MB 53.3MB/s 
     |████████████████████████████████| 901kB 71.3MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [2]:
%%capture
!apt-get install maven -qq

In [3]:
%%capture
!git clone --recurse-submodules https://github.com/castorini/anserini.git
%cd anserini
!cd tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!mvn clean package appassembler:assemble -DskipTests -Dmaven.javadoc.skip=true

In [4]:
!ls target

anserini-0.11.1-SNAPSHOT-fatjar.jar  classes		     maven-archiver
anserini-0.11.1-SNAPSHOT.jar	     generated-sources	     maven-status
appassembler			     generated-test-sources  test-classes


In [5]:
%%capture
!wget https://git.uwaterloo.ca/jimmylin/anserini-indexes/raw/master/index-robust04-20191213.tar.gz
!tar xvfz index-robust04-20191213.tar.gz -C indexes/

In [6]:
!du -h indexes/index-robust04-20191213

2.1G	indexes/index-robust04-20191213


In [7]:
%%script bash
cd tools/topics-and-qrels/
ls

docids.covid.round1.txt
docids.covid.round2.txt
docids.covid.round3.txt
docids.covid.round4.txt
docids.covid.round5.txt
prels.mq.10001-20000.txt
prels.mq.1-10000.txt
prels.mq.20001-60000.txt
prels.web.1-50.txt
qrels.adhoc.101-150.txt
qrels.adhoc.151-200.txt
qrels.adhoc.401-450.txt
qrels.adhoc.451-550.txt
qrels.adhoc.51-100.txt
qrels.backgroundlinking18.txt
qrels.backgroundlinking19.txt
qrels.cacm.txt
qrels.car17v1.5.benchmarkY1test.txt
qrels.car17v2.0.benchmarkY1test.txt
qrels.clef06fr.txt
qrels.core17.txt
qrels.core18.txt
qrels.covid-complete.txt
qrels.covid-round12.txt
qrels.covid-round1.txt
qrels.covid-round2-cumulative.txt
qrels.covid-round2.txt
qrels.covid-round3-cumulative.txt
qrels.covid-round3.txt
qrels.covid-round4-cumulative.txt
qrels.covid-round4.txt
qrels.covid-round5.txt
qrels.dl19-doc.txt
qrels.dl19-passage.txt
qrels.dl20-doc.txt
qrels.dl20-passage.txt
qrels.fire12bn.176-225.txt
qrels.fire12en.176-225.txt
qrels.fire12hi.176-225.txt
qrels.microblog2011.txt
qrels.microblog2

In [8]:
%%script bash
python -m pyserini.search --topics robust04 \
 --index indexes/index-robust04-20191213 \
 --output runs/run.robust04.bm25tuned.txt \
 --bm25 --hits 100 --k1 0.9 --b 0.4

Setting BM25 parameters: k1=0.9, b=0.4
Running robust04 topics, saving to runs/run.robust04.bm25tuned.txt...


100%|██████████| 250/250 [00:04<00:00, 59.48it/s]


In [12]:
%%script bash
python -m pyserini.search --topics robust04 \
 --index indexes/index-robust04-20191213 \
 --output runs/run.robust04.bm25tuned.txt \
 --bm25 --hits 100 

Running robust04 topics, saving to runs/run.robust04.bm25tuned.txt...


100%|██████████| 250/250 [00:05<00:00, 48.89it/s]


In [13]:
%%script bash
cd runs/
ls

run.robust04.bm25tuned.txt


In [10]:
%%script bash
cd indexes/index-robust04-20191213/
ls

_h.fdt
_h.fdx
_h.fnm
_h_Lucene50_0.doc
_h_Lucene50_0.pos
_h_Lucene50_0.tim
_h_Lucene50_0.tip
_h_Lucene80_0.dvd
_h_Lucene80_0.dvm
_h.nvd
_h.nvm
_h.si
_h.tvd
_h.tvx
segments_2
write.lock


In [14]:
!tools/eval/trec_eval.9.0.4/trec_eval -c -mrecall.100 -mmap -mndcg tools/topics-and-qrels/qrels.robust04.txt runs/run.robust04.bm25tuned.txt

map                   	all	0.2154
recall_100            	all	0.4137
ndcg                  	all	0.3909
